In [ ]:
using CairoMakie
using Arya
using LilGuys

In [ ]:
using OrderedCollections
using Measurements
import TOML
using CSV, DataFrames

In [ ]:
using Printf

In [ ]:
Makie.set_theme!(Arya.theme_arya(), 
    fontsize = 24,
    Axis = (;
        xgridvisible=false,
        ygridvisible = false,
        ),
)

# Observational Density Profiles

In [ ]:
scl_obs_prof = LilGuys.StellarProfile(ENV["DWARFS_ROOT"] * "/observations/sculptor/density_profiles/" * "fiducial" * "_profile.toml")
umi_obs_prof = LilGuys.StellarProfile(ENV["DWARFS_ROOT"] * "/observations/ursa_minor/density_profiles/" * "fiducial" * "_profile.toml")

In [ ]:
import SciPy
import NaNMath as nm

In [ ]:
function predict_properties(Σ_model; N=10_000, log_r_min=-2, log_r_max=2)
    log_r_bins = LinRange(log_r_min, log_r_max, 1000)
    log_r = midpoints(log_r_bins)
    r = 10 .^  log_r
    r_bins = 10 .^ log_r_bins
    
    Σ = Σ_model.(r)

    #Γ = calc_Γ(log_r, Σ)
    M_in = [LilGuys.integrate(rrr->2π*rrr*Σ_model(rrr), 0, rr) for rr in r]
    Σ_m = M_in ./ (π * r .^ 2)
    Γ_max = 2*(1 .- Σ ./ Σ_m)
    counts =  Σ .* (2π *  r .* diff(r_bins) )

    #Γ=Γ, 
    return (log_r=log_r, log_r_bins=log_r_bins, counts=counts, Σ=Σ, Σ_m=Σ_m, Γ_max=Γ_max, M_in=M_in)

end

In [ ]:
import LinearAlgebra: diag

In [ ]:
function fit_profile(obs; r_max=r_max, N=10_000, profile=lguys.Exp2D, p0=[2, 0.3], kwargs...)
    r_val = 10 .^ obs.log_r
    log_Σ = obs.log_Sigma .± obs.log_Sigma_err
    filt = r_val .< r_max
    filt .&= map(x->isfinite(x), log_Σ)
    filt .&= @. !isnan(log_Σ)
    
    r_val = r_val[filt]
    log_Σ = log_Σ[filt]


    log_Σ_val = [s.val for s in log_Σ]
    log_Σ_e = [s.err for s in log_Σ]

	log_Σ_exp(r, popt...) = nm.log10.(LilGuys.calc_Σ.(profile(popt...), r))
	popt, covt = SciPy.optimize.curve_fit(log_Σ_exp, r_val, log_Σ_val, 
        sigma=log_Σ_e, p0=p0; kwargs...)
    
    popt_p = popt .± sqrt.(diag(covt))

	Σ_pred(r) = 10 .^ log_Σ_exp(r, popt...)
    props = predict_properties(Σ_pred, 
        N=N, log_r_min=obs.log_r_bins[1], log_r_max=obs.log_r_bins[end])

    log_Σ_pred = log_Σ_exp.(10 .^ obs.log_r, popt...)
    log_Σ_res = Measurements.value.(obs.log_Sigma).- log_Σ_pred
    return popt_p, props, log_Σ_res
end

In [ ]:
popt, pred = fit_profile(scl_obs_prof, p0=[10_000, 7], profile=LilGuys.Exp2D, r_max=20)

In [ ]:
pred.

In [ ]:
profile = scl_obs_prof
fig = Figure()


ylabel=L"\log \Sigma\ /\ \textrm{stars arcmin^{-2}}"


ax = Axis(fig[1, 1],
    ylabel=ylabel,
    xlabel=log_r_label,
    title = "Sculptor"
)

errscatter!(ax, profile.log_r, profile.log_Sigma, 
                yerr=profile.log_Sigma_err)
lines!(ax, pred.log_r, log10.(pred.Σ), color=COLORS[2])

fig

In [ ]:
popt, pred = fit_profile(umi_obs_prof, p0=[10_000, 7], profile=LilGuys.Exp2D, r_max=20)

In [ ]:
profile = umi_obs_prof
fig = Figure()


ylabel=L"\log \Sigma\ /\ \textrm{stars arcmin^{-2}}"


ax = Axis(fig[1, 1],
    ylabel=ylabel,
    xlabel=log_r_label,
    title = "Ursa Minor",
    limits=(nothing, nothing, -4, nothing)
)

errscatter!(ax, profile.log_r, profile.log_Sigma, 
                yerr=profile.log_Sigma_err)

lines!(ax, pred.log_r, log10.(pred.Σ), color=COLORS[2])

fig

In [ ]:
function load_obs_profiles(filenames, galaxy="sculptor")

    profiles = OrderedDict{String, LilGuys.StellarProfile}()

    for (key, filename) in filenames
        profiles[key] = LilGuys.StellarProfile(ENV["DWARFS_ROOT"] * "/observations/$galaxy/density_profiles/" * filename * "_profile.toml")

    end
    
    return profiles
end

In [ ]:
log_r_label = "log r / arcmin"

In [ ]:
function plot_obs_densities(profiles; sequential=false,
        normalize=false,
		limits=((-1, 2), (-3, 2))
    )
   	fig = Figure()

    if normalize
		ylabel=L"\log \Sigma\ /\ \textrm{fraction arcmin^{-2}}"
    else
        ylabel=L"\log \Sigma\ /\ \textrm{stars arcmin^{-2}}"
    end
    
    ax = Axis(fig[1, 1],
		ylabel=ylabel,
		xlabel=log_r_label,
        limits = limits,
    )

    for (i, (key, profile)) in enumerate(profiles)
        if sequential
            kwargs = (; color=i, colorrange=(1, length(profiles)))
        else
            kwargs = (; )
        end

        if normalize
            dy = -log10(profile.M_in[end])
        else
            dy = 0
        end
        
        lines!(ax, profile.log_r, profile.log_Sigma .+ dy;
            label=key, kwargs...)
        
        if i == 1
            errscatter!(ax, profile.log_r, profile.log_Sigma .+ dy, 
                yerr=profile.log_Sigma_err)
        end
    end

    axislegend(position=:lb)

    fig
end

In [ ]:
function plot_Γ(profiles)
   	fig = Figure()
	ax = Axis(fig[1, 1],
		ylabel=L"\Gamma",
		xlabel=log_r_label,
		limits=((-1, 2), (-5, 2))
	)

    for (i, (key, profile)) in enumerate(profiles)
        lines!(ax, profile.log_r, profile.Gamma, 
                label=key)

        
        if i == 1
            errscatter!(ax, profile.log_r, profile.Gamma, 
                yerr=profile.Gamma_err)

        end
    end

    axislegend()

    fig
end

In [ ]:
function plot_Γ_max(profiles)
   	fig = Figure()
	ax = Axis(fig[1, 1],
		ylabel=L"\Gamma_\textrm{max}",
		xlabel=log_r_label,
		limits=((-1, 2), (-2, 3))
	)

    for (i, (key, profile)) in enumerate(profiles)
        
        lines!(ax, profile.log_r, profile.Gamma_max, 
            label=key)

        if i == 1
            errscatter!(ax, profile.log_r, profile.Gamma_max,
                yerr=profile.Gamma_max_err)
        end

    end

    axislegend(position=:rb)

    fig
end

### Basic uncertainties

In [ ]:
profiles = load_obs_profiles(OrderedDict(
    "PSAT > 0.2" => "fiducial",
    "simple cuts" => "simple",
    "no qso cand." => "noqso",
    "circular radii" => "psat_circ",
    "DELVE" => "delve",
        ))

In [ ]:
plot_obs_densities(profiles, normalize=true, limits=(-1, 2.5, -7, -1.5))

In [ ]:
plot_Γ(profiles)

In [ ]:
plot_Γ_max(profiles)

In [ ]:
prof = profiles["PSAT > 0.2"]

for i in eachindex(prof.log_r)
    @printf "%6.2f %8.2f ± %8.2f\n" prof.log_r[i] prof.Gamma_max[i] prof.Gamma_max_err[i]
end

In [ ]:
profiles = load_obs_profiles(OrderedDict(
    "PSAT > 0.2" => "fiducial",
    "simple cuts" => "simple",
    "no qso cand." => "noqso",
    "circular radii" => "psat_circ",
        ), "ursa_minor")

In [ ]:
prof = profiles["PSAT > 0.2"]

for i in eachindex(prof.log_r)
    @printf "%6.2f %8.2f ± %8.2f\n" prof.log_r[i] prof.Gamma_max[i] prof.Gamma_max_err[i]
end

In [ ]:
plot_obs_densities(profiles, normalize=true, limits=(-1, 2.5, -7, -1.5))

In [ ]:
plot_Γ(profiles)

In [ ]:
plot_Γ_max(profiles)

## Fit the profiles

# Observational background

# Radial velocities

In [ ]:
memb_stars = LilGuys.read_fits(ENV["DWARFS_ROOT"] * "/observations/sculptor/processed/sculptor_memb_rv.fits"
);

In [ ]:
let
	fig, ax = FigAxis(
		xlabel = L"\xi / \textrm{degree}",
		ylabel = L"\eta / \textrm{degree}",
		aspect=DataAspect(),
		xreversed=true,
	)

	w = 1 ./ memb_stars.vz_err

	bins = (33, 25)
	k1 = Arya.histogram2d(memb_stars.xi,memb_stars.eta, bins, weights= w .* memb_stars.vz)
	k2 = Arya.histogram2d(memb_stars.xi,memb_stars.eta, bins, weights=w)

	k1.values ./= k2.values

	p = heatmap!(k1.xbins, k1.ybins, k1.values, 
		colormap=:bluesreds,
		colorrange=(50, 100)
		)
	Colorbar(fig[1, 2], p, label="GSR radial velocity / km/s",
)

	fig
end

# model (nbody) Orbits

In [ ]:
"""
    load_orbit(filename)

"""
function load_orbit(filename)
    out = Output(joinpath(modelsdir, filename))

    df = DataFrame(
        time = out.times * T2GYR, 
        x = out.x_cen[1, :],
        y = out.x_cen[2, :],
        z = out.x_cen[3, :],
        v_x = out.v_cen[1, :] * V2KMS,
        v_y = out.v_cen[2, :] * V2KMS,
        v_z = out.v_cen[3, :] * V2KMS,
        )


    return out.times, out.x_cen, out.v_cen
end

In [ ]:
function load_csv_orbit(orbit_file)
    	orbit_expected = CSV.read(orbit_file, DataFrame)
    	x_cen_exp = transpose(hcat(orbit_expected.x, orbit_expected.y, orbit_expected.z))
    	v_cen_exp = transpose(hcat(orbit_expected.v_x, orbit_expected.v_y, orbit_expected.v_z))

        return orbit_expected.time, x_cen_exp, v_cen_exp
end

In [ ]:
modelname = "sculptor/1e6_V31_r4.2/vasiliev+21_smallperi"

In [ ]:
lmc_file = joinpath(modelsdir, modelname, "lmc_traj.csv")

In [ ]:
orbit_scl = load_orbit(joinpath(modelsdir, modelname))

In [ ]:
orbit_lmc = load_csv_orbit(lmc_file)

In [ ]:
idx_gyr = [argmin(abs.(orbit_model[1] .- i / T2GYR )) for i in -5:0]

In [ ]:
LilGuys.Plots.plot_xyz(orbit_model[2], orbit_lmc[2], 
    labels=["Scl", "LMC"],
    idx_scatter=fill(idx_gyr[end:end], 2)
)



In [ ]:
using GeoMakie

In [ ]:
using SkyCoords

In [ ]:
function get_l_b(positions)
    gc = [Galactocentric(x=pos[1], y=pos[2], z=pos[3], v_x=NaN, v_y=NaN, v_z=NaN) for pos in eachcol(positions)]
    icrs = LilGuys.transform.(ICRS, gc)

    sc = [ICRSCoords(deg2rad(o.ra), deg2rad(o.dec)) for o in icrs]

    gal = convert.(GalCoords, sc)

    l = [g.l for g in gal]
    b = [g.b for g in gal]
    r = [o.distance for o in icrs]
    return rad2deg.(l), rad2deg.(b), r
end

In [ ]:
r_scl_lmc = calc_r(orbit_model[2], orbit_lmc[2])

In [ ]:
idx_perilmc = argmin(r_scl_lmc[300:end]) + 299

In [ ]:
let
    fig = Figure()
	ax = Axis(fig[1,1], xlabel="time / Gyr", ylabel = "distance / kpc",
        limits=(nothing, nothing, 0, nothing)
    )
    lines!(orbit_model[1] * T2GYR, calc_r(orbit_model[2]), label="Scl - MW")

    lines!(orbit_model[1] * T2GYR, r_scl_lmc, label="Scl - LMC")

    #lines!(orbit_model[1] * T2GYR, calc_r(orbit_lmc[2]), label="Scl - LMC")

	axislegend(ax, position=:lb)
	fig
end

In [ ]:
Makie.available_marker_symbols()

In [ ]:
let 
	fig = Figure()
    gridcolor = xgridcolor=RGBA(0., 0., 0., 0.1)
    
	ax = GeoAxis(fig[1,1]; dest = "+proj=aitoff", 
        xgridcolor=gridcolor, ygridcolor=gridcolor
    )

    l, b, r = get_l_b(orbit_model[2])
    lines!(l, b)
    scatter!(l[idx_gyr], b[idx_gyr], label="Sculptor")
    scatter!(l[idx_gyr[end]], b[idx_gyr[end]], color=COLORS[1], marker=:xcross, markersize=20)
    scatter!(l[idx_perilmc], b[idx_perilmc], color=COLORS[1], marker=:star5, markersize=20)

    l, b,r  = get_l_b(orbit_lmc[2])
    lines!(l, b)

    scatter!(l[idx_gyr], b[idx_gyr], label="LMC")
    scatter!(l[idx_gyr[end]], b[idx_gyr[end]], color=COLORS[2], marker=:xcross, markersize=20)
    scatter!(l[idx_perilmc], b[idx_perilmc], color=COLORS[2], marker=:star5, markersize=20)

	fig
end

In [ ]:
orbits = [
    "mean" => load_orbit("sculptor/1e7_V31_r3.2/orbit_mean"),
    "smallperi" => load_orbit("sculptor/1e7_V31_r3.2/orbit_smallperi"),
    #"heavy" => load_orbit("sculptor/1e6_V40_r5.9/orbit_mean"),
    ];

In [ ]:
compare_orbits_2d(orbits, yname="r", xname="time", aspect=nothing, xunits="Gyr", limits=(0, 10, 35, 110))

In [ ]:
compare_orbits_2d(orbits, yname="z", xname="y", limits=(-100,100,-100,100))

# MC Orbits

In [ ]:
using Printf

In [ ]:
using StatsBase
using HDF5

In [ ]:
function read_traj(name)
    local positions, velocities, times
    
    h5open(joinpath(modelsdir, "sculptor/mc_orbits/", "$name/trajectory.hdf5"), "r") do f
        positions = f["positions"][:, :, :]
        velocities = f["velocities"][:, :, :]
        times = -f["times"][:]
    end

    return positions, velocities, times
end

In [ ]:
function read_distribution(name)
    return LilGuys.read_fits(joinpath(modelsdir, "sculptor/mc_orbits/", name, "peris_apos.fits"))
end

In [ ]:
function compare_peris(families)
    
	fig = Figure()
	ax = Axis(fig[1, 1],
		xlabel = "pericentre / kpc",
		ylabel = "pdf"
	)

    for (label, df) in families
    	bins, counts, err = LilGuys.histogram(df.pericentre, normalization=:pdf)
    	lines!(midpoints(bins), counts, label=label)
    end

    axislegend()

	fig
end

In [ ]:
function compare_t_last_peris(families)
    
	fig = Figure()
	ax = Axis(fig[1, 1],
		xlabel = "time since pericentre / Gyr",
		ylabel = "pdf"
	)

    for (label, df) in families
    	bins, counts, err = LilGuys.histogram(df.t_last_peri, normalization=:pdf)
    	lines!(midpoints(bins), counts, label=label)
    end

    axislegend()

	fig
end

In [ ]:
function compare_apos(families)
    
	fig = Figure()
	ax = Axis(fig[1, 1],
		xlabel = "apocentre / kpc",
		ylabel = "pdf"
	)

    for (label, df) in families
    	bins, counts, err = LilGuys.histogram(df.apocentre, normalization=:pdf)
    	lines!(midpoints(bins), counts, label=label)
    end

    axislegend()

	fig
end

In [ ]:
function compare_stats(families)

    for (label, df) in families
        @printf "%16s%12.2f [%0.2f %0.2f] [[%0.2f %0.2f]]\n" label quantile(df.pericentre, [0.5, 0.16, 0.84, 0.0014, 0.9986])...
    end
end

In [ ]:
families = [
    "EP20 (MW only)" => read_distribution("systematic_errors"),
    "V+21 (MW only)" => read_distribution("vasiliev_nolmc"),
    "V+21 (MW+LMC)" => read_distribution("vasiliev_lmc"),
    ]

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1],
    xlabel = "time since pericentre / Gyr",
    ylabel = "pericentre",
    limits=(0, nothing, 0, nothing)
)

for (label, df) in families
    scatter!(df.t_last_peri*T2GYR .+ 0.2*T2GYR * randn(length(df.t_last_peri)), df.pericentre, label=label => (; markersize=10), markersize=3, alpha=0.1)
end

df = families[end].second
scatter!(df.t_last_peri_lmc * T2GYR, df.peri_lmc, label="V+21 (LMC frame)" =>(; markersize=10), markersize=3, alpha=0.1)


axislegend(position=:lt)

fig

In [ ]:
traj = read_traj("vasiliev_lmc")

In [ ]:
traj_no = read_traj("vasiliev_nolmc")

In [ ]:
V_T2GYR = 0.97779

In [ ]:
# loads in trajectory of lmc in Vasiliev 2021
lmc_file = ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev+21/trajlmc.txt"
lmc_traj = CSV.read(lmc_file, DataFrame, delim=" ", header = [:time, :x, :y, :z, :v_x, :v_y, :v_z])

lmc_x = LilGuys.lerp(lmc_traj.time, lmc_traj.x)
lmc_y = LilGuys.lerp(lmc_traj.time, lmc_traj.y)
lmc_z = LilGuys.lerp(lmc_traj.time, lmc_traj.z)
lmc_v_x = LilGuys.lerp(lmc_traj.time, lmc_traj.v_x)
lmc_v_y = LilGuys.lerp(lmc_traj.time, lmc_traj.v_y)
lmc_v_z = LilGuys.lerp(lmc_traj.time, lmc_traj.v_z)

times_v = traj[3] * T2GYR / V_T2GYR
pos = reshape([lmc_x.(times_v) lmc_y.(times_v) lmc_z.(times_v)]', (3, 1, :))
vel = reshape([lmc_v_x.(times_v) lmc_v_y.(times_v) lmc_v_z.(times_v)]', (3, 1, :))

In [ ]:
function plot_r_t_traj!(traj; alpha=0.01, color=:black, kwargs...)
    positions, velocities, times = traj
    for i in 1:size(positions, 2)
        x = times * T2GYR
        y = calc_r(positions[:, i, :])
        lines!(x, y; alpha=alpha, color=color, kwargs...)
    
    end
end

In [ ]:
function plot_x_y_traj!(traj; alpha=0.01, color=:black, kwargs...)
    positions, velocities, times = traj
    for i in 1:size(positions, 2)
        x = positions[2, i, :]
        y = positions[3, i, :]
        
        lines!(x, y; alpha=alpha, color=color, kwargs...)
    
    end
end

In [ ]:
traj_lmc = pos, vel, traj[3]
traj_scl_lmc = pos .- traj[1], vel .- traj[2], traj[3]


In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], xlabel="time / Gyr", ylabel = "radius / kpc",
    xgridvisible=false, ygridvisible=false
)

plot_r_t_traj!(traj, label="Scl-MW", color=COLORS[1])
plot_r_t_traj!(traj_scl_lmc, label="Scl-LMC", color=COLORS[2])
plot_r_t_traj!(traj_lmc, label="LMC-MW", color=:black, alpha=1, linewidth=3, )


axislegend(unique=true)

Makie.save("figures/scl_lmc_mw_r_t_samples.pdf", fig)

fig

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], xlabel="y / kpc", ylabel="z / kpc",
    xgridvisible=false, ygridvisible=false, 
    aspect=DataAspect(),
)
plot_x_y_traj!(traj_no, label="Scl, MW only")
plot_x_y_traj!(traj, label="Scl, MW + LMC", color=COLORS[1])
plot_x_y_traj!(traj_lmc, label="LMC", alpha=1, color=COLORS[2], linewidth=3)

axislegend(unique=true)

fig

# Circular velocity profiles

In [ ]:
function load_profile(name) 
    path = joinpath(models_dir, "$name/profiles.hdf5")
    profiles = LilGuys.read_structs_from_hdf5(path, LilGuys.MassProfile3D)
    idx = parse.(Int, first.(profiles))
    out = LilGuys.Output(joinpath(models_dir, name))
        
    return out.times[sort(idx)], profiles[sortperm(idx)]
    
end

In [ ]:
function compare_vcirc(profiles; errskip=1, legend=true)
    xlims = (-2, 3)
    
    fig = Figure()
    ax = Axis(fig[1, 1],
        xlabel=LP.log_r_label,
        ylabel=L"$\log\,v_\textrm{circ}$ / km\,s$^{-1}$",
#        limits=(xlims[1], xlims[2], -0.1, 1.7),
        )
    pi = 1

    
    for i in eachindex(profiles)
        label, profs = profiles[i]
        x = [prof.r_circ_max for (_, prof) in profs]
        y = [prof.v_circ_max for (_, prof) in profs]
        lines!(log10.(x), log10.(y*V2KMS), label=label)
    end
    
    for i in eachindex(profiles)
        label, profs = profiles[i]
        x = [prof.r_circ_max for (_, prof) in profs]
        y = [prof.v_circ_max for (_, prof) in profs]
        scatter!(log10.(x[end]), log10.(y[end]*V2KMS))
    end
    if legend
        axislegend(position=:lt)
    end
    fig
end

In [ ]:
function compare_vcirc(profiles; errskip=1, legend=true)
    xlims = (-2, 3)
    
    fig = Figure()
    ax = Axis(fig[1, 1],
        xlabel=LP.log_r_label,
        ylabel=L"$\log\,v_\textrm{circ}$ / km\,s$^{-1}$",
#        limits=(xlims[1], xlims[2], -0.1, 1.7),
        )
    pi = 1

    
    for i in eachindex(profiles)
        label, profs = profiles[i]
        x = [prof.r_circ_max for (_, prof) in profs]
        y = [prof.v_circ_max for (_, prof) in profs]
        lines!(log10.(x), log10.(y*V2KMS), label=label)
    end
    
    for i in eachindex(profiles)
        label, profs = profiles[i]
        x = [prof.r_circ_max for (_, prof) in profs]
        y = [prof.v_circ_max for (_, prof) in profs]
        scatter!(log10.(x[end]), log10.(y[end]*V2KMS))
    end
    if legend
        axislegend(position=:lt)
    end
    fig
end

In [ ]:
models_dir = "/astro/dboyea/dwarfs/analysis/sculptor"

In [ ]:
dm_profiles = [
    "mean" => load_profile("1e7_V31_r3.2/orbit_mean"),
    "smallperi" => load_profile("1e7_V31_r3.2/orbit_smallperi"),
    "LMC" => load_profile("1e6_V31_r4.2/vasiliev+21_smallperi"),
    "heavy LMC" => load_profile("1e6_V31_r3.2/vasiliev+21_heavylmc_smallperilmc"),
    ];

In [ ]:

fig = Figure()
ax = Axis(fig[1, 1],
    xlabel="time / Gyr",
    ylabel=L"$\log\,v_\textrm{circ}$ / km\,s$^{-1}$",
    )
pi = 1


for i in eachindex(dm_profiles)
    label, (x, profs) = dm_profiles[i]
    y = [prof.v_circ_max for (_, prof) in profs]
    lines!((x*T2GYR), log10.(y*V2KMS), label=label)
end


fig

# Stellar velocity dispersion

In [ ]:
function load_profile(modelname, starsname)
    orbit = load_profiles(modelname * "/stars/" * starsname)

    return orbit
end

In [ ]:
function load_isolation(modelname, starsname)
    iso = load_profiles(modelname * "/../stars/" * starsname)
    return iso
end

In [ ]:
function load_profiles(filename; extension="stellar_profiles_3d.hdf5")
    path = joinpath(modelsdir, filename, extension)
    
    profs = LilGuys.read_structs_from_hdf5(path, LilGuys.StellarProfile3D)
    idxs = parse.(Int, first.(profs))
    profs = last.(profs)

    s = sortperm(idxs)
    return [idxs[i] => profs[i] for i in s ]
end

In [ ]:
function load_prof_expected(filename, parampath="profile.toml")
    path = joinpath(modelsdir, filename, parampath)
        
    if isfile(parampath)
        expected = LilGuys.load_profile(parampath)
    else
        error("file not found $path")
    end

    return expected
end

In [ ]:
function compare_profiles(profiles, expected=nothing; 
        sequential=false, legend=true, limits=(-1.5, 0.8, -15, 3), 
        kwargs...
    )
    
    fig = Figure()

	ax = Axis(fig[1,1], xlabel=L"\log\, r / \textrm{kpc}", ylabel =  L"\log\, \rho_\star\; [10^{10} M_\odot / \textrm{kpc}^3]", 
		limits=limits; kwargs...
		)

    plot_kwargs = Dict{Symbol, Any}()

    if sequential
        plot_kwargs[:colorrange] = (1, length(profiles))
    end
    
    for i in eachindex(profiles)
        label, profs = profiles[i]
        prof = profs[end].second
        x = prof.log_r
        y = log10.(prof.rho)
        if sequential
            plot_kwargs[:color] = i
        end

    	lines!(x, y, label=string(label); plot_kwargs...)
        arrows!([log10.(prof.r_break)],[-10], [0], [3], color=COLORS[i])
    end

    if expected !== nothing
    	log_r_pred = LinRange(-2, 2, 1000)
    	ρ_s_pred = calc_ρ.(expected, 10 .^ log_r_pred)

    	lines!(log_r_pred, log10.(ρ_s_pred), label="expected", color="black", linestyle=:dot)
    end

    if legend
    	axislegend(ax, position=:lb)
    end 
    
	fig
end

In [ ]:
function plot_v_t!(profiles; x_shift = 0, kwargs...)
	sigmas = [p.second.sigma_vx for p in profiles]

    t = [p.second.time for p in profiles] 
	lines!(t * T2GYR .+ x_shift, sigmas * V2KMS; kwargs...)
end

In [ ]:
function plot_v_t(profiles; x_shift=0, kwargs...)
	fig, ax = FigAxis(
		xlabel = "time / Gyr",
		ylabel = L"\sigma_v / \textrm{km s^{-1}}";
        kwargs...
	)

    plot_v_t!(profiles, x_shift=x_shift)

    fig
end
    

In [ ]:
function compare_v_t(profilesi, profs_iso=nothing; x_shifts=zeros(length(profilesi)), legend_position=:lb, kwargs...)
	fig, ax = FigAxis(
		xlabel = "time / Gyr",
		ylabel = L"\sigma_v / \textrm{km s^{-1}}",
	)


    for i in eachindex(profilesi)
        label, profiles = profilesi[i]

        kwargs = Dict{Symbol, Any}()
        kwargs[:label] = label
        kwargs[:color] = COLORS[i]
  
        plot_v_t!(profiles; x_shift=x_shifts[i], kwargs...)
    end

    if profs_iso !== nothing
        for i in eachindex(profs_iso)
            label, profiles = profs_iso[i]
    
      
            sigmas = [p.second.sigma_vx for p in profiles]
    
            t = [p.second.time for p in profiles] 
            t = t .- t[end]
            lines!(t * T2GYR, sigmas * V2KMS; color=COLORS[i], kwargs...)
            
        end
    end
    
    hlines!(obs_props["sigma_v"], color=:grey)
    hspan!(obs_props["sigma_v"] - obs_props["sigma_v_err"], 
        obs_props["sigma_v"] + obs_props["sigma_v_err"],
        color = (:grey, 0.1), label="observed")

    LilGuys.Plots.hide_grid!(ax)
    if legend_position == :outside
        Legend(fig[1,2], ax)
    else
        
        axislegend(position=legend_position)
    end
    
	fig
end

In [ ]:
profilesi = [
    "mean" => load_profile("sculptor/1e7_V31_r3.2/orbit_mean", "exp2d_rs0.13"),
    "smallperi" => load_profile("sculptor/1e7_V31_r3.2/orbit_smallperi", "exp2d_rs0.08"),
    "LMC" => load_profile("sculptor/1e6_V31_r4.2/vasiliev+21_smallperi", "exp2d_rs0.13"),
    "heavy LMC" => load_profile("sculptor/1e6_V31_r3.2/vasiliev+21_heavylmc_smallperilmc", "exp2d_rs0.13"),
    ];

In [ ]:
import TOML
obs_props = TOML.parsefile(joinpath(ENV["DWARFS_ROOT"], "observations/sculptor/observed_properties.toml"))

In [ ]:
compare_v_t(profs, x_shifts=[-10, -10, 0,0,0 ], legend_position=:lb)

In [ ]:

fig = Figure(size=(700, 400))
ax = Axis(fig[1, 1],
    xlabel="time / Gyr",
    ylabel=L"$v$ / km\,s$^{-1}$",
    yscale=log10
    )
pi = 1
x_shifts=[-10, -10, 0,0,0 ]

for i in eachindex(profilesi)
    label, profiles = profilesi[i]

    kwargs = Dict{Symbol, Any}()
    #kwargs[:label] = label
    kwargs[:color] = COLORS[i]
    kwargs[:linestyle] = :dash

    plot_v_t!(profiles; x_shift=x_shifts[i], kwargs...)
end



for i in eachindex(dm_profiles)
    label, (x, profs) = dm_profiles[i]
    y = [prof.v_circ_max for (_, prof) in profs]
    lines!((x*T2GYR .+ x_shifts[i]), (y*V2KMS), label=label)
end

hlines!(obs_props["sigma_v"], color=:grey)
hspan!(obs_props["sigma_v"] - obs_props["sigma_v_err"], 
    obs_props["sigma_v"] + obs_props["sigma_v_err"],
    color = (:grey, 0.1), label=L"observed $\sigma_v$")


lines!([NaN], [NaN], color=:black, label=L"DM $v_\textrm{max}$")
lines!([NaN], [NaN], color=:black, label=L"Stellar $\sigma_v$", linestyle=:dash)

ax.yticks =  [7, 10, 20, 30, 40]

Legend(fig[1,2], ax)
fig

# Model density profiles

In [ ]:
function sigma_axis(gp; kwargs...) 
	ax = Axis(gp, 
		xlabel=log_r_label,
		ylabel = L"\log \Sigma\ / \textrm{(stars/arcmin^2)}",
        limits=(-1, 2, -2, 2.5)
		;kwargs...
	)

	return fig, ax
end

In [ ]:
prof_expected = LilGuys.StellarProfile("/astro/dboyea/dwarfs/observations/sculptor/density_profiles/fiducial_profile.toml");



In [ ]:
log_r_label = "log r / arcmin"
log_sigma_label = L"$\log \Sigma$ / stars arcmin$^{-2}$"

In [ ]:
function plot_finals(models)
    
    fig = Figure()
    Axis(fig[1, 1], 
        xlabel = log_r_label, 
        ylabel = log_sigma_label,
        limits = (-0.5, 2.3, -2, 2)
        )

	errscatter!(prof_expected.log_r, prof_expected.log_Sigma,
		yerr=prof_expected.log_Sigma_err,
		color=:black,
        label="J+24",
	)

    for (label, model) in models
    	lines!(model.log_r, model.log_Sigma, 
    			label=label)
    end

	axislegend()

	fig
end

In [ ]:
function plot_ini_fin(profs, model, normalization=0; 
        limits=(-0.5, 2.3, -2, 2),
        r_b = NaN,
        kwargs...
    )
    
    fig = Figure()
    Axis(fig[1, 1];
        xlabel = log_r_label, 
        ylabel = log_sigma_label,
        limits = limits, 
        kwargs...
    )

	errscatter!(prof_expected.log_r, prof_expected.log_Sigma,
		yerr=prof_expected.log_Sigma_err,
		color=:black,
        label="J+24",
	)

    for (label, prof) in profs
        lines!(prof.log_r, prof.log_Sigma .+ normalization, 
                label=label)
    end


    if model !== nothing

        x = LinRange(-2, 2.5, 1000)
        r = 10 .^ x
        y = LilGuys.calc_Σ.(model, r)
    
        r_scale = LilGuys.kpc_to_arcmin(1, profs[1].second.distance)
    
    
        x = x .+ log10(r_scale)
        y .*= 1 / r_scale ^ 2
    
        lines!(x, log10.(y) .+ normalization, linestyle=:dot, color=:black, label="exponential")

    end
    
    if r_b !== NaN
        dy = limits[4]-limits[3]
        dx = limits[2] - limits[1]
        
        x0 = limits[1]
        y0 = limits[3]

        a = log10(r_b)
        _, prof = profs[end]
        idx = argmin(abs.(prof.log_r .- a))
        b = prof.log_Sigma[idx] + normalization .+ 0.1dy
        va = 0
        vb = -0.05dy
        
        arrows!([a], [b], [va], [vb])
        println(a, ", ", b)
        text!(a, b, text=L"r_b")

    end

	axislegend(position=:lb)


	fig
end

In [ ]:
function mean_groups_with_uncertainties(data::Vector{T}, uncertainties::Vector{T}, n::Int) where T
    num_full_groups = div(length(data), n)
    remainder = length(data) % n
    
    # Create arrays to store the means and their uncertainties
    means = Vector{T}(undef, num_full_groups + (remainder > 0 ? 1 : 0))
    uncertainties_mean = Vector{T}(undef, length(means))
    
    # Function to compute weighted mean and uncertainty of the mean
    function weighted_mean(data_group, uncertainty_group)
        weights = 1 ./ (uncertainty_group .^ 2)
        m = sum(weights .* data_group) / sum(weights)
        uncertainty_of_mean = sqrt(1 / sum(weights))
        return (m, uncertainty_of_mean)
    end

    # Calculate the means and uncertainties of full groups
    for i in 1:num_full_groups
        start_index = (i - 1) * n + 1
        end_index = i * n
        means[i], uncertainties_mean[i] = weighted_mean(data[start_index:end_index], uncertainties[start_index:end_index])
    end
    
    # Handle the remaining data, if any
    if remainder != 0
        start_index = num_full_groups * n + 1
        means[end], uncertainties_mean[end] = weighted_mean(data[start_index:end], uncertainties[start_index:end])
    end
    
    return means, uncertainties_mean
end


In [ ]:
function plot_ini_fin_slope(profs, model; 
        binsize=3,
        obs_binsize=2,
        limits=(-0.5, 2.3, -2, 2),
        r_b = NaN,
        figsize=(600, 300),
        kwargs...
    )
    
    fig = Figure(size=figsize)
    ax = Axis(fig[1, 1];
        xlabel = log_r_label, 
        ylabel = L"\Gamma",
        limits = limits, 
        kwargs...
    )

    ax_lin = Axis(fig[1, 2];
        xlabel = "r / arcmin", 
        ylabel = L"\Gamma",
        limits = (0, 10^limits[2], limits[3], limits[4]), 
        kwargs...
    )

    x = prof_expected.log_r
    y = prof_expected.Gamma
    ye = prof_expected.Gamma_err

    x = mean_groups(x, obs_binsize)
    y, ye = mean_groups_with_uncertainties(y, ye, obs_binsize)
    
	errscatter!(ax, x, y,
		yerr=ye,
		color=:black,
        label="J+24",
	)


    errscatter!(ax_lin, 10 .^ x, y,
		yerr=ye,
		color=:black,
	)

    for (label, prof) in profs
        x = mean_groups(prof.log_r, binsize)
        y = mean_groups(prof.Gamma, binsize)
        lines!(ax, x, y, label=label)
        lines!(ax_lin, 10 .^ x, y)
    end


    if model !== nothing

        x = LinRange(-2, 2.5, 1000)
        r = 10 .^ x
        y = LilGuys.calc_Σ.(model, r)
    
        r_scale = LilGuys.kpc_to_arcmin(1, profs[1].second.distance)
    
    
        x = x .+ log10(r_scale)
        y .*= 1 / r_scale ^ 2
        gamma = LilGuys.gradient(log10.(y), x)
    
        lines!(ax, x, gamma, linestyle=:dot, color=:black, label="exponential")
        lines!(ax_lin, 10 .^ x, gamma, linestyle=:dot, color=:black, label="exponential")

    end
    
    if r_b !== NaN
        dy = limits[4]-limits[3]
        dx = limits[2] - limits[1]
        
        x0 = limits[1]
        y0 = limits[3]

        a = log10(r_b)
        _, prof = profs[end]
        idx = argmin(abs.(prof.log_r .- a))
        b = prof.Gamma[idx] .+ 0.1dy
        va = 0
        vb = -0.05dy
        
        arrows!(ax, [a], [b], [va], [vb])
        println(a, ", ", b)
        text!(ax, a, b, text=L"r_b")

    end


    linkyaxes!(ax, ax_lin)
    hideydecorations!(ax_lin, grid=false, ticks=false)
    
	#axislegend(ax, position=:lb)
    resize_to_layout!(fig)

	fig
end

In [ ]:
import StatsBase: mean

In [ ]:
function mean_groups(data::AbstractVector{T}, n::Int) where T
    num_full_groups = div(length(data), n)
    remainder = length(data) % n
    
    # Create an array to store the means
    means = Vector{T}(undef, num_full_groups + (remainder > 0 ? 1 : 0))
    
    # Calculate the means of full groups
    for i in 1:num_full_groups
        start_index = (i - 1) * n + 1
        end_index = i * n
        means[i] = mean(data[start_index:end_index])
    end
    
    # Handle the remaining data, if any
    if remainder != 0
        start_index = num_full_groups * n + 1
        means[end] = mean(data[start_index:end])
    end
    
    return means
end

In [ ]:
mean_groups(1:10, 3)

In [ ]:
function get_log_Σ_0(prof, χ2_max=1)

    local log_Σ_0, log_Σ_0_err

    pre_filt = isfinite.(prof.log_Sigma)
    log_r = prof.log_r[pre_filt]
    log_Σ = prof.log_Sigma[pre_filt]
    log_Σ_err = prof.log_Sigma_err[pre_filt]
    @assert issorted(log_r)

    for i in eachindex(log_r)
        filt = 1:i
        log_Σ_0 = LilGuys.mean(log_Σ[filt] .± log_Σ_err[filt])
        log_Σ_0_err = Measurements.uncertainty.(log_Σ_0)
        log_Σ_0 = Measurements.value(log_Σ_0)
        χ2 = @. (log_Σ_0 - log_Σ)^2 / (log_Σ_err + log_Σ_0_err)^2

        χ2_norm = LilGuys.mean(χ2[filt])

        if χ2_norm .> χ2_max
            @info "max radius of $(log_r[i])"
            break
        end
    end
    
    return log_Σ_0, log_Σ_0_err
end

In [ ]:
function get_normalization(prof_f, prof_expected=prof_expected)
    log_Σ_0_obs, _ = get_log_Σ_0(prof_expected)
    log_Σ_0, _ = get_log_Σ_0(prof_f)

    return log_Σ_0_obs - log_Σ_0
end

In [ ]:
modelsdir = ENV["DWARFS_ROOT"] * "/analysis/"

In [ ]:
function read_profiles(filename, starsname)
    prof_i = LilGuys.StellarProfile(modelsdir * filename * "/stars/$starsname/initial_profile.toml")
    prof_f = LilGuys.StellarProfile(modelsdir * filename * "/stars/$starsname/final_profile.toml")
    prof_ana = LilGuys.load_profile(modelsdir * filename * "/../stars/$starsname/profile.toml")

    
    profs = [
        "initial" => prof_i,
        "present day" => prof_f,
    ]

    return profs, prof_ana
end

## LMC

In [ ]:
modeldir = "sculptor/1e6_V31_r4.2/vasiliev+21_smallperi"

In [ ]:
readdir(joinpath(modelsdir, modeldir, "stars/exp2d_rs0.13"))

In [ ]:
orbit_props = TOML.parsefile(joinpath(modelsdir, modeldir, "orbital_properties.toml"))

In [ ]:
profs, expected = read_profiles(modeldir, "exp2d_rs0.13");

In [ ]:
prof_f = profs[end].second
norm = get_normalization(prof_f)

r_b = LilGuys.calc_break_radius(prof_f.sigma_v / V2KMS, 0.10 / T2GYR)
r_b_arcmin = LilGuys.kpc_to_arcmin(r_b, orbit_props["distance_f"])

In [ ]:
plot_ini_fin(profs, expected, norm, limits=(-1, 3, -6, 2), r_b=r_b_arcmin)

In [ ]:
plot_ini_fin_slope(profs, expected, limits=(-1, 2, -10, 4), binsize=5, r_b=r_b_arcmin)

In [ ]:
modeldir = "sculptor/1e6_V31_r3.2/vasiliev+21_heavylmc_smallperilmc"

In [ ]:
readdir(joinpath(modelsdir, modeldir, "stars/plummer_rs0.20"))

In [ ]:
orbit_props = TOML.parsefile(joinpath(modelsdir, modeldir, "orbital_properties.toml"))

In [ ]:
profs, expected = read_profiles(modeldir, "exp2d_rs0.13");

In [ ]:
prof_f = profs[end].second
norm = get_normalization(prof_f)

r_b = LilGuys.calc_break_radius(prof_f.sigma_v / V2KMS, 0.10 / T2GYR)
r_b_arcmin = LilGuys.kpc_to_arcmin(r_b, orbit_props["distance_f"])

In [ ]:
plot_ini_fin(profs, expected, norm, limits=(-1, 3, -6, 2), r_b=r_b_arcmin)

In [ ]:
plot_ini_fin_slope(profs, expected, limits=(-1, 2, -10, 4), binsize=5, r_b=r_b_arcmin)

### 1e7

In [ ]:
modeldir = "sculptor/1e7_V31_r3.2/orbit_smallperi"

In [ ]:
orbit_props = TOML.parsefile(joinpath(modelsdir, modeldir, "orbital_properties.toml"))

In [ ]:
profs, expected = read_profiles(modeldir, "exp2d_rs0.08");

In [ ]:
prof_f = profs[end].second
norm = get_normalization(prof_f)

r_b = LilGuys.calc_break_radius(prof_f.sigma_v / V2KMS,  orbit_props["t_last_peri"]/ T2GYR)
r_b_arcmin = LilGuys.kpc_to_arcmin(r_b, orbit_props["distance_f"])

In [ ]:
plot_ini_fin(profs, expected, norm, limits=(-1, 3, -6, 2), 
    r_b=r_b_arcmin, title="MW only, small pericentre"
)

In [ ]:
plot_ini_fin_slope(profs, expected, limits=(-1, 2, -10, 4), 
    binsize=5, r_b=r_b_arcmin, 
    )

In [ ]:
profs, expected = read_profiles(modeldir, "plummer_rs0.18");

In [ ]:
prof_f = profs[end].second
norm = get_normalization(prof_f)

r_b = LilGuys.calc_break_radius(prof_f.sigma_v / V2KMS,  orbit_props["t_last_peri"]/ T2GYR)
r_b_arcmin = LilGuys.kpc_to_arcmin(r_b, orbit_props["distance_f"])

In [ ]:
plot_ini_fin(profs, expected, norm, limits=(-1, 3, -6, 2), 
    r_b=r_b_arcmin, title="Plummer",
)

In [ ]:
plot_ini_fin_slope(profs, expected, limits=(-1, 2, -10, 4), 
    binsize=5, r_b=r_b_arcmin, 
    )

In [ ]:
modeldir = "sculptor/1e7_V31_r3.2/orbit_mean"

In [ ]:
orbit_props = TOML.parsefile(joinpath(modelsdir, modeldir, "orbital_properties.toml"))

In [ ]:
profs, expected = read_profiles(modeldir, "exp2d_rs0.13");

In [ ]:
prof_f = profs[end].second
norm = get_normalization(prof_f)

r_b = LilGuys.calc_break_radius(prof_f.sigma_v / V2KMS,  orbit_props["t_last_peri"]/ T2GYR)
r_b_arcmin = LilGuys.kpc_to_arcmin(r_b, orbit_props["distance_f"])

In [ ]:
plot_ini_fin(profs, expected, norm, limits=(-1, 3, -6, 2), r_b=r_b_arcmin, title="MW only, mean orbit")

In [ ]:
plot_ini_fin_slope(profs, expected, limits=(-1, 2, -10, 4), binsize=5, r_b=r_b_arcmin)

# Gamma max

In [ ]:
readdir(modelsdir * "/isolation/1e7/fiducial/")

In [ ]:
snap_0 = LilGuys.Snapshot(modelsdir * "/isolation/1e7/fiducial/combined.hdf5/1")

In [ ]:
r = @. sqrt(snap_0.positions[1, :]^2 + snap_0.positions[2, :]^2)

In [ ]:
bins=LilGuys.Interface.bins_both(r, nothing; bin_width=0.05, num_per_bin=1000)

In [ ]:
LilGuys.Interface.default_n_per_bin(r)

In [ ]:
prof_3d = LilGuys.MassProfile3D(snap_0)

In [ ]:

lines(prof_3d.log_r, log10.(prof_3d.rho),
    axis=(; limits=(nothing, nothing, -8, 0))
)

In [ ]:
bins=LilGuys.Interface.bins_both(log10.(r), nothing, num_per_bin=6_000, bin_width=0.05)

In [ ]:
prof = LilGuys.StellarProfile(r, bins=bins)

In [ ]:
lines(prof.log_r, prof.log_Sigma)

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], limits=(nothing, 3.5, 0, 5))
lines!(prof.log_r, -prof.Gamma)

fig

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], limits=(nothing, 3.5, -2, 5))
lines!(prof.log_r, prof.Gamma_max)

fig